In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import numpy as np
import cv2
import copy
import math
def remove_edge_cells(mask_image):
    w,h = mask_image.shape
    pruned_mask = copy.deepcopy(mask_image)
    remove_list = []
    edges = mask_image[0,:],mask_image[w-1,:],mask_image[:,0],mask_image[:,h-1]
    for edge in edges:
        edge_masks = np.unique(edge)
        for edge_mask in edge_masks:
            remove_list.append(edge_mask)
            pruned_mask[np.where(mask_image==edge_mask)] = 0

    return pruned_mask


def remove_small_cells(mask_image,area_threshold=10000):
    w,h = mask_image.shape
    pruned_mask = copy.deepcopy(mask_image)
    for mask_index in np.unique(mask_image):
        # if mask_index == mask_image[330,640]:
        #     a = 1
        area = np.sum(mask_image == mask_index)
        if area < area_threshold:
            pruned_mask[np.where(mask_image == mask_index)] = 0


    return pruned_mask

def split_cell_masks(mask_image):
    # Convert the mask image to grayscale
    gray_mask = cv2.cvtColor(np.array(mask_image*255,dtype=np.uint8), cv2.COLOR_BGR2GRAY)

    # Find unique RGB values in the mask image
    unique_colors = np.unique(mask_image.reshape(-1, mask_image.shape[2]), axis=0)

    # Create an empty index mask
    index_mask = np.zeros_like(gray_mask)

    # Assign unique index values to each cell in the index mask
    for i, color in enumerate(unique_colors):
        # Find pixels with the current color in the mask image
        color_mask = np.all(mask_image == color, axis=2)

        # Assign the index value to the corresponding pixels in the index mask
        index_mask[color_mask] = i + 1

    index_mask[np.where(gray_mask==255)]=0

    return index_mask


def remove_concentric_masks(mask_image):
    # Convert the mask image to grayscale
    cell_values = np.unique(mask_image)
    for i in range(1, len(cell_values)):# remove background
        mask_one = np.array(mask_image == cell_values[i],dtype=np.uint8)
        # mask_one_dilated = cv2.dilate(mask_one, np.ones((5, 5), np.uint8),100)
        # xmin, xmax, ymin, ymax = np.min(np.where(mask_one == 1)[0]), np.max(np.where(mask_one == 1)[0]),\
        #     np.min(np.where(mask_one == 1)[1]), np.max(np.where(mask_one == 1)[1]),
        contour, _ = cv2.findContours(mask_one, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if len(contour) > 0:
            largest_contour = max(contour, key=cv2.contourArea)

            mask_image = cv2.drawContours(mask_image, [largest_contour], -1, (int(cell_values[i])), thickness=cv2.FILLED)



    return mask_image



def analyze_cell_properties(mask):
    properties = {}

    # Calculate properties from the mask
    contour, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_idx = np.argmax([len(contour[i]) for i in range(len(contour))])


    perimeter = cv2.arcLength(contour[contour_idx], True)

    area = cv2.contourArea(contour[contour_idx])

    _, radius = cv2.minEnclosingCircle(contour[contour_idx])

    ellipse = cv2.fitEllipse(contour[contour_idx])
    ellipse_contour = cv2.ellipse2Poly((int(ellipse[0][0]), int(ellipse[0][1])),
                                       (int(ellipse[1][0] * 0.5), int(ellipse[1][1] * 0.5)),
                                       int(ellipse[2]), 0, 360, 5)

    # Compute the perimeter of the fitted ellipse
    perimeter_ellipse = cv2.arcLength(ellipse_contour, closed=True)

    # Compute the smoothness as the ratio of perimeters
    smoothness = perimeter_ellipse / perimeter

    compactness = abs((perimeter ** 2) / (area * 4 * math.pi) - 1)

    symmetry = cv2.matchShapes(contour[contour_idx], cv2.convexHull(contour[contour_idx], returnPoints=True), 1, 0.0)

    # Store the properties
    properties['perimeter'] = perimeter
    properties['area'] = area
    properties['radius'] = radius
    properties['non-smoothness'] = smoothness
    properties['non-circularity'] = compactness
    # properties['concavity'] = concavity
    properties['symmetry'] = symmetry

    return properties



In [6]:
%cd "/content/drive/MyDrive/Final IDCC-SAM/"

!git clone https://github.com/facebookresearch/segment-anything.git
!cd segment-anything; pip install -e.

!pip install git+https://github.com/facebookresearch/segment-anything.git

/content/drive/MyDrive/Final IDCC-SAM
fatal: destination path 'segment-anything' already exists and is not an empty directory.
Obtaining file:///content/drive/MyDrive/Final%20IDCC-SAM/segment-anything
  Preparing metadata (setup.py) ... done
  Running setup.py develop for segment-anything
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-avmrmqyg
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-avmrmqyg
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36590 sha256=f2bba3401dd7eb3c078562562425ba08dddb0337854fbc3cc35fb07d6f8f5c18
  Stored in directory: /tmp/pip-ephem-wheel-cache-xtrsl082/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built

In [7]:
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry,  SamAutomaticMaskGenerator, SamPredictor

In [8]:
if not os.path.exists("/content/drive/MyDrive/Final IDCC-SAM/segment-anything/sam_vit_h_4b8939.pth"):
  !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -O /content/drive/MyDrive/Final IDCC-SAM/segment-anything/sam_vit_h_4b8939.pth

In [9]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

# Path to the SAM model checkpoint, downloaded during installation
sam_checkpoint = "/content/drive/MyDrive/Final IDCC-SAM/segment-anything/sam_vit_h_4b8939.pth"

#Use the visual transformer backbone
model_type = "vit_h"

device = "cuda"

# Load the SAM model from the registry using the provided checkpoint
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)



mask_generator = SamAutomaticMaskGenerator(sam)



def combine_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    # ax = plt.gca()
    # ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        xmin,ymin,xmax,ymax = ann['bbox']

        if abs(xmin-0) <5 and abs(ymin-0) <5 and abs(xmax-sorted_anns[0]['segmentation'].shape[0]) <5 \
                and abs(ymax-sorted_anns[0]['segmentation'].shape[1]) <5:
            pass
        else:
            color_mask = np.concatenate([np.random.random(3), [0.35]])
            img[m] = color_mask
    # ax.imshow(img)
    return img


In [10]:
import numpy as np
import cv2

def patch_property(image, start_idx):
    masks = mask_generator.generate(image)

    # Combine the annotations generated from SAM
    final_mask = combine_anns(masks)

    # Indexing the objects in the segmentation mask
    index_mask = split_cell_masks(final_mask)

    pruned_mask = remove_edge_cells(index_mask)
    pruned_mask_reduce = remove_small_cells(pruned_mask, area_threshold=1500)
    pruned_mask_reduce = remove_concentric_masks(pruned_mask_reduce)
    cell_mask = np.zeros((pruned_mask.shape[0], pruned_mask.shape[1], 3), dtype=np.uint8)

    cell_num = len(np.unique(pruned_mask_reduce)) - 1
    properties = {}
    for i in range(1, cell_num + 1):  # Remove background
        mask_one = np.array(pruned_mask_reduce == np.unique(pruned_mask_reduce)[i], dtype=np.uint8)
        try:
            properties['cell %i' % (i + start_idx)] = analyze_cell_properties(mask_one)
            cell_color = (np.random.randint(255), np.random.randint(255), np.random.randint(255))
            contours, hierarchy = cv2.findContours(mask_one, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            cv2.drawContours(cell_mask, contours, -1, cell_color, 3)

            text = str(i + start_idx)
            # Define the font properties
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 1
            font_color = (0, 0, 0)  # Black color (BGR format)
            thickness = 2

            # Calculate the position to center the text on the image
            text_x = (np.max(np.where(mask_one == 1)[0]) + np.min(np.where(mask_one == 1)[0])) // 2
            text_y = (np.max(np.where(mask_one == 1)[1]) + np.min(np.where(mask_one == 1)[1])) // 2

            # Draw the text on a separate mask
            text_mask = np.zeros_like(cell_mask)
            text_mask = cv2.putText(text_mask, text, (text_y, text_x), font, font_scale, font_color, thickness)

            # Add the text mask to the cell mask
            cell_mask = cv2.addWeighted(cell_mask, 1, text_mask, 1, 0)
        except ZeroDivisionError:
            pass

    # Blend the cell mask with the original image
    cell_mask = cv2.addWeighted(cell_mask, 1, image, 1, 0)
    return properties, cell_num + start_idx, cell_mask


In [11]:
import time
#from analysis import patch_property
import os
from tifffile import imread
import argparse
import matplotlib
import cv2
import pandas as pd

fpath = "/content/drive/MyDrive/Final IDCC-SAM/Dataset/Rebuttal/IDCIA/Test_clahe/images"
output_path = '/content/drive/MyDrive/Final IDCC-SAM/Rebuttal/IDCIA/SAM4Organoid/Test results_SAM4Organoid'
csv_output_path = '/content/drive/MyDrive/Final IDCC-SAM/Rebuttal/IDCIA/SAM4Organoid/Test results_cell counts'

os.makedirs(output_path, exist_ok=True)
os.makedirs(csv_output_path, exist_ok=True)


for file in os.listdir(fpath):
    if '.png' in file:
        fn = os.path.join(fpath,file)
        cell_properties = {}
        sta_time = time.time()
        cell_properties = {}
        try:
            image = cv2.imread(fn)
        except:
            image = imread(fn)

        # image preprocessing
        w, h, _ = image.shape
        w, h = int((w - 10000) / 2), int((h - 10000) / 2)

        # calculate properties
        cell_properties_patch,sta_idx, cell_mask_batch = patch_property(image,0)
        cell_properties.update(cell_properties_patch)

        # save segmentation mask and cell properties
        cv2.imwrite(os.path.join(output_path,file),
        #cv2.imwrite(os.path.join(output_path,file.strip('.png') +  '_cell_mask.png'),

        cell_mask_batch)

        cell_properties = pd.DataFrame.from_dict(cell_properties,orient='index')
        cell_properties.to_csv(os.path.join(csv_output_path,
                                           file.replace('.png','.csv')))

        end_time = time.time()
        print('File %s completed, time used %0.2f min'%(fn,(end_time-sta_time)/60))

File /content/drive/MyDrive/Final IDCC-SAM/Dataset/Rebuttal/IDCIA/Test_clahe/images/220909_GFP-AHPC_D_MAP2ab_F1_Cy3_ND2_20x.png completed, time used 0.25 min
File /content/drive/MyDrive/Final IDCC-SAM/Dataset/Rebuttal/IDCIA/Test_clahe/images/220909_GFP-AHPC_D_MAP2ab_F2_Cy3_ND2_20x.png completed, time used 0.22 min
File /content/drive/MyDrive/Final IDCC-SAM/Dataset/Rebuttal/IDCIA/Test_clahe/images/220909_GFP-AHPC_D_MAP2ab_F4_Cy3_ND2_20x.png completed, time used 0.20 min
File /content/drive/MyDrive/Final IDCC-SAM/Dataset/Rebuttal/IDCIA/Test_clahe/images/220909_GFP-AHPC_D_MAP2ab_F5_Cy3_ND2_20x.png completed, time used 0.18 min
File /content/drive/MyDrive/Final IDCC-SAM/Dataset/Rebuttal/IDCIA/Test_clahe/images/220909_GFP-AHPC_D_MAP2ab_F7_Cy3_ND2_20x.png completed, time used 0.18 min
File /content/drive/MyDrive/Final IDCC-SAM/Dataset/Rebuttal/IDCIA/Test_clahe/images/220909_GFP-AHPC_D_MAP2ab_F9_Cy3_ND2_20x.png completed, time used 0.16 min
File /content/drive/MyDrive/Final IDCC-SAM/Dataset/R

Summarize Prediction CSVs

In [12]:
import os
import pandas as pd

# Define the folder path containing the CSV files
folder_path = '/content/drive/MyDrive/Final IDCC-SAM/Rebuttal/IDCIA/SAM4Organoid/Test results_cell counts'

# Initialize a list to store the summary data
summary_data = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a CSV file
    if filename.lower().endswith('.csv'):
        # Read the CSV file
        csv_path = os.path.join(folder_path, filename)
        df = pd.read_csv(csv_path)

        # Count the number of non-empty cells in the first column
        non_empty_count = df.iloc[:, 0].dropna().shape[0]

        # Prepare the image name (replace .csv with .png)
        image_name = filename.replace('.csv', '.png')

        # Append the data to the summary list
        summary_data.append([image_name, non_empty_count])

# Create a DataFrame for the summary data
summary_df = pd.DataFrame(summary_data, columns=['Image Name', 'Prediction'])

# Load the ground_truth.csv file
ground_truth_path = '/content/drive/MyDrive/Final IDCC-SAM/Dataset/Rebuttal/IDCIA/ground_truth.csv'
ground_truth_df = pd.read_csv(ground_truth_path)

# Merge the summary DataFrame with the ground truth DataFrame
merged_df = summary_df.merge(ground_truth_df, left_on='Image Name', right_on='image name')

# Drop the extra 'image name' column after merge
merged_df.drop(columns=['image name'], inplace=True)
merged_df.drop(columns=['SN'], inplace=True)

# Calculate the Raw Absolute Error
merged_df['Raw Absolute Error'] = (merged_df['Prediction'] - merged_df['ground truth']).abs()

# Check if Raw Absolute Error is <= 10 and create the 'AAE <=10' column
merged_df['AAE <=10'] = merged_df['Raw Absolute Error'].apply(lambda x: 'Yes' if x <= 10 else 'No')


# Save the merged DataFrame to a new Excel file
summary_excel_path = '/content/drive/MyDrive/Final IDCC-SAM/Rebuttal/IDCIA/SAM4Organoid/SAM4Organoid_prediction_summary_IDCIA.xlsx'
merged_df.to_excel(summary_excel_path, index=False)

print("Summary Excel file with ground truth has been created.")


Summary Excel file with ground truth has been created.
